In [ ]:
from ModelTools.Metric import Metric
from ModelTools.RegBuilder import RegBuilder
from ModelTools._src import tabulate
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
rng = np.random.default_rng(0)
x = rng.normal(0,0.1,100).cumsum()
y = rng.normal(3+2*x+x**2,scale=0.3)
df = pd.DataFrame({'x':x,'y':y})

m=RegBuilder('poly_OLS')
m.fit(x.reshape(-1,1),y)

In [ ]:
m.coef

In [ ]:
import matplotlib.pyplot as plt

x_new = np.linspace(-3,3,100)
y_new = rng.normal(3+2*x_new+x_new**2,scale=0.3)

mean = m.predict(x_new.reshape(-1,1))
lw,up=m.predict_interval(x_new.reshape(-1,1),alpha=0.01)

plt.plot(x_new,mean,c='red')
plt.scatter(x_new,y_new)
plt.fill_between(x_new,lw,up,alpha=0.3)
plt.vlines(x=min(x),ymin=min(lw),ymax=max(up),colors='green')
plt.vlines(x=max(x),ymin=min(lw),ymax=max(up),colors='green')

In [ ]:
m.intercept

In [ ]:
m.feature_names_in

In [ ]:
list(m.feature_names)